# Automated ML



In [1]:
#pip install -r /anaconda/envs/azureml_py36/lib/python3.6/site-packages/azureml/automl/core/validated_linux_requirements.txt

In [2]:
import pandas as pd
## Azure-related
import azureml.core
print("SDK version:", azureml.core.VERSION)

SDK version: 1.20.0


##  Create compute cluster
        # Get WS details and Subscription details

In [3]:
from azureml.core import Workspace, Experiment, Run
#ws = Workspace.get(name="udacity-project")
ws = Workspace.from_config() 

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

Workspace name: quick-starts-ws-136346
Azure region: southcentralus
Subscription id: a24a24d5-8d87-4c8a-99b6-91ed2d2df51f
Resource group: aml-quickstarts-136346


## Create Cluster

In [4]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
import os

# choose a name for your cluster
compute_name = os.environ.get("AML_COMPUTE_CLUSTER_NAME", "std-d2-v2-vn")
compute_min_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MIN_NODES", 0)
compute_max_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MAX_NODES", 4)

# This example uses CPU VM. For using GPU VM, set SKU to STANDARD_NC6
vm_size = os.environ.get("AML_COMPUTE_CLUSTER_SKU", "STANDARD_D2_V2")


if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        print('found compute target. just use it. ' + compute_name)
else:
    print('creating a new compute target...')
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = vm_size,
                                                                min_nodes = compute_min_nodes, 
                                                                max_nodes = compute_max_nodes)

    # create the cluster
    compute_target = ComputeTarget.create(ws, compute_name, provisioning_config)
    
    # can poll for a minimum number of nodes and for a specific timeout. 
    # if no min node count is provided it will use the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    
     # For a more detailed view of current AmlCompute status, use get_status()
    print(compute_target.get_status().serialize())

## Dataset

### Overview
* The incident management log data is retrieved from UCI Machine learnig data set. [Incident Data)[https://archive.ics.uci.edu/ml/datasets/Incident+management+process+enriched+event+log]. The data is sourced from a Servicenow platform for IT service management.  
  
* This provides details of various incidents recorded over period of time . I have taken up this project to predict ETA(expected time of accomplishment) to understand time to resolve each incident. This helps IT department to provide ETA for customers based on time taken to resolve similar issues historically. This will also help IT department to understand if any instance will go beyond expected SLA.


from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

In [5]:
import pandas as pd
import numpy as np
import os
import zipfile
from zipfile import ZipFile
from zipfile import BadZipFile
from glob import iglob
from io import BytesIO
from urllib.request import urlopen
import dateutil.parser

from sklearn import datasets, linear_model
from time import time
import datetime
from datetime import timedelta
import time

import argparse
import joblib
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.ensemble import GradientBoostingRegressor

from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core.run import Run
from math import sqrt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import LabelEncoder
import source.train as src


In [6]:

def loadData(ds):
    #datasource = 'incident_event_log.zip'
    # datasource = urlopen("https://archive.ics.uci.edu/ml/machine-learning-databases/00498/incident_event_log.zip")
    datasource = urlopen(ds)
    try:
        # with zipfile.ZipFile('incident_event_log.zip') as ds:
        ds = ZipFile(BytesIO(datasource.read()))
        ds.extractall()
        ds.namelist()
        #df = pd.read_csv(zipfile.open('incident_event_log.csv')
        df = pd.read_csv(next(iglob('*.csv')))
        return(df)  
    except zipfile.BadZipFile:
        print('Bad zip file')

In [7]:

import pandas as pd

datasource = "https://archive.ics.uci.edu/ml/machine-learning-databases/00498/incident_event_log.zip"
df = src.loadData()

#df.head(2)
df.head()


,number,incident_state,active,reassignment_count,reopen_count,sys_mod_count,made_sla,caller_id,opened_by,opened_at,...,u_priority_confirmation,notify,problem_id,rfc,vendor,caused_by,closed_code,resolved_by,resolved_at,closed_at
0,INC0000045,New,True,0,0,0,True,Caller 2403,Opened by 8,29/2/2016 01:16,...,False,Do Not Notify,?,?,?,?,code 5,Resolved by 149,29/2/2016 11:29,5/3/2016 12:00
1,INC0000045,Resolved,True,0,0,2,True,Caller 2403,Opened by 8,29/2/2016 01:16,...,False,Do Not Notify,?,?,?,?,code 5,Resolved by 149,29/2/2016 11:29,5/3/2016 12:00
2,INC0000045,Resolved,True,0,0,3,True,Caller 2403,Opened by 8,29/2/2016 01:16,...,False,Do Not Notify,?,?,?,?,code 5,Resolved by 149,29/2/2016 11:29,5/3/2016 12:00
3,INC0000045,Closed,False,0,0,4,True,Caller 2403,Opened by 8,29/2/2016 01:16,...,False,Do Not Notify,?,?,?,?,code 5,Resolved by 149,29/2/2016 11:29,5/3/2016 12:00
4,INC0000047,New,True,0,0,0,True,Caller 2403,Opened by 397,29/2/2016 04:40,...,False,Do Not Notify,?,?,?,?,code 5,Resolved by 81,1/3/2016 09:52,6/3/2016 10:00


In [8]:
df.dtypes

number                     object
incident_state             object
active                       bool
reassignment_count          int64
reopen_count                int64
sys_mod_count               int64
made_sla                     bool
caller_id                  object
opened_by                  object
opened_at                  object
sys_created_by             object
sys_created_at             object
sys_updated_by             object
sys_updated_at             object
contact_type               object
location                   object
category                   object
subcategory                object
u_symptom                  object
cmdb_ci                    object
impact                     object
urgency                    object
priority                   object
assignment_group           object
assigned_to                object
knowledge                    bool
u_priority_confirmation      bool
notify                     object
problem_id                 object
rfc           

## Clean data 
## Craete features and target variable

In [9]:

def clean_data(df):
    # select Incidents that are closed 
    df_closed = df[df['incident_state']=='Closed']
    df1 = df_closed[['incident_state','category','assignment_group','reopen_count',
                     'made_sla','impact','urgency','priority',
                     'opened_at','closed_at','closed_code','caused_by']]
    
    list1 = []
    for val in df1['opened_at']:
        d1 = datetime.datetime.strptime(val, "%d/%m/%Y %H:%M")
        t1 = time.mktime(d1.timetuple())
        list1.append(t1)
        
    list2 = []
    for val in df1['closed_at']:
        d2 = datetime.datetime.strptime(val, "%d/%m/%Y %H:%M")
        t2 = time.mktime(d2.timetuple())
        list2.append(t2)
        
    list3 = []
    for i in range(len(list1)):
        list3.append(list2[i]-list1[i])
        
    df1['time_to_close'] = np.array(list3)
    df1['time_to_close'] = df1['time_to_close']/3600.0
    
    priority=pd.get_dummies(df1.priority, prefix ="priority")
    df1.drop('priority', inplace=True, axis=1)
    df1= df1.join(priority)
    
    impact=pd.get_dummies(df1.impact, prefix ="impact")
    df1.drop('impact', inplace=True, axis=1)
    df1 = df1.join(impact)
    
    urgency=pd.get_dummies(df1.urgency, prefix ="urgency")
    df1.drop('urgency', inplace=True, axis=1)
    df1 = df1.join(urgency)
    
    closed_code=pd.get_dummies(df1.closed_code, prefix ="closed_code")
    df1.drop('closed_code', inplace=True, axis=1)
    df1 = df1.join(closed_code)
    
    lb_make = LabelEncoder()
    df1['category'] = lb_make.fit_transform(df1['category'])
    df1['assignment_group'] = lb_make.fit_transform(df1['assignment_group'])
    
    # df1['active']= df1['active'].astype(int)
    df1['made_sla']=df1['made_sla'].astype(int)
    
    df1 = df1.dropna()
    # df = pd.get_dummies(df)
    #df1['time_to_close'] = time_to_close
    df1 =df1.drop(['incident_state','opened_at','closed_at','caused_by'],axis=1)
    #x = df1
    #X = df1.drop(['time_to_close'],axis=1)
    y= df1['time_to_close']
    return(df1, y)



In [10]:
### Get Test and Train split

df1, y = clean_data(df)

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_

In [11]:
df1.columns

Index(['category', 'assignment_group', 'reopen_count', 'made_sla',
       'time_to_close', 'priority_1 - Critical', 'priority_2 - High',
       'priority_3 - Moderate', 'priority_4 - Low', 'impact_1 - High',
       'impact_2 - Medium', 'impact_3 - Low', 'urgency_1 - High',
       'urgency_2 - Medium', 'urgency_3 - Low', 'closed_code_?',
       'closed_code_code 1', 'closed_code_code 10', 'closed_code_code 11',
       'closed_code_code 12', 'closed_code_code 13', 'closed_code_code 14',
       'closed_code_code 15', 'closed_code_code 16', 'closed_code_code 17',
       'closed_code_code 2', 'closed_code_code 3', 'closed_code_code 4',
       'closed_code_code 5', 'closed_code_code 6', 'closed_code_code 7',
       'closed_code_code 8', 'closed_code_code 9'],
      dtype='object')

In [12]:
# X_train, X_test, y_train, y_test = src.train_test_data(X,y)

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=11)

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [13]:
import logging

automl_settings = {
    "iteration_timeout_minutes": 10,
    "experiment_timeout_hours": 0.3,
    "enable_early_stopping": True,
    "primary_metric": 'r2_score',
    "featurization": 'auto',
    "verbosity": logging.INFO,
    "n_cross_validations": 5
}


In [14]:
# df1['time_to_close']

In [15]:
# TODO: Put your automl config here
from azureml.train.automl import AutoMLConfig

automl_config = AutoMLConfig(task='regression',
                             debug_log='automated_ml_errors.log',
                             experiment_timeout_minutes=20,
                             primary_metric='r2_score',
                             training_data=df1,
                             label_column_name='time_to_close',
                             featurization='auto',
                             verbosity=logging.INFO,
                             n_cross_validations=3)
                          


## Submit AUTO ML Experiment

In [16]:
# TODO: Submit your experiment

from azureml.core.experiment import Experiment
experiment = Experiment(ws, 'incidents-exp')
automl_run = experiment.submit(automl_config)


#run = experiment.submit(automl_config, show_output=True)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [17]:
# submit an experiment
from azureml.widgets import RunDetails

automl_run = experiment.submit(automl_config, show_output=True)
RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output=True)




Running on local machine
Parent Run ID: AutoML_5b4f94d5-e680-469e-890b-774eb1fb462b

Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardi

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

DURATION: Time taken for the current iteration.
METRIC: The result of computing score on the fitted pipeline.
BEST: The best observed score thus far.
****************************************************************************************************

 ITERATION   PIPELINE                                       DURATION      METRIC      BEST
         0   MaxAbsScaler LightGBM                          0:00:29       0.4922    0.4922
         1   MaxAbsScaler XGBoostRegressor                  0:00:29       0.4613    0.4922
         2   SparseNormalizer XGBoostRegressor              0:00:42       0.4554    0.4922
         3   MaxAbsScaler RandomForest                      0:01:08       0.3148    0.4922
         4   StandardScalerWrapper XGBoostRegressor         0:00:39       0.4789    0.4922
         5   MaxAbsScaler ElasticNet                        0:00:39       0.2022    0.4922
         6   MaxAbsScaler RandomForest                      0:01:47       0.2841    0.4922
         7   Standar

{'runId': 'AutoML_5b4f94d5-e680-469e-890b-774eb1fb462b',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2021-01-28T06:43:16.91796Z',
 'endTimeUtc': '2021-01-28T07:04:59.074367Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'r2_score',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '3',
  'target': 'local',
  'AMLSettingsJsonString': '{"path":null,"name":"incidents-exp","subscription_id":"a24a24d5-8d87-4c8a-99b6-91ed2d2df51f","resource_group":"aml-quickstarts-136346","workspace_name":"quick-starts-ws-136346","region":"southcentralus","compute_target":"local","spark_service":null,"azure_service":"Microsoft.AzureNotebookVM","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"r2_score","task_type":"regression","data_script":null,"validation_size":0.0,"n_cross_validations":3,"y_min":null,"y_max":null,"num_classes":null,"featu

In [18]:
best_run, fitted_model = automl_run.get_output()
print(best_run)
print(fitted_model)

Run(Experiment: incidents-exp,
Id: AutoML_5b4f94d5-e680-469e-890b-774eb1fb462b_22,
Type: None,
Status: Completed)
RegressionPipeline(pipeline=Pipeline(memory=None,
                                     steps=[('datatransformer',
                                             DataTransformer(enable_dnn=None,
                                                             enable_feature_sweeping=None,
                                                             feature_sweeping_config=None,
                                                             feature_sweeping_timeout=None,
                                                             featurization_config=None,
                                                             force_text_dnn=None,
                                                             is_cross_validation=None,
                                                             is_onnx_compatible=None,
                                                             logger=None,
   

Best Run Model on another meric

In [19]:
lookup_metric = "root_mean_squared_error"
best_run, fitted_model = automl_run.get_output(metric = lookup_metric)
print(best_run)
print(fitted_model)

Run(Experiment: incidents-exp,
Id: AutoML_5b4f94d5-e680-469e-890b-774eb1fb462b_22,
Type: None,
Status: Completed)
RegressionPipeline(pipeline=Pipeline(memory=None,
                                     steps=[('datatransformer',
                                             DataTransformer(enable_dnn=None,
                                                             enable_feature_sweeping=None,
                                                             feature_sweeping_config=None,
                                                             feature_sweeping_timeout=None,
                                                             featurization_config=None,
                                                             force_text_dnn=None,
                                                             is_cross_validation=None,
                                                             is_onnx_compatible=None,
                                                             logger=None,
   

In [20]:
fitted_model.steps[-1]

('prefittedsoftvotingregressor',
 PreFittedSoftVotingRegressor(estimators=[('0',
                                           Pipeline(memory=None,
                                                    steps=[('maxabsscaler',
                                                            MaxAbsScaler(copy=True)),
                                                           ('lightgbmregressor',
                                                            LightGBMRegressor(boosting_type='gbdt',
                                                                              class_weight=None,
                                                                              colsample_bytree=1.0,
                                                                              importance_type='split',
                                                                              learning_rate=0.1,
                                                                              max_depth=-1,
                      

In [21]:
#Exporting most optimal model obtained from AutoML using joblib
import joblib
joblib.dump(fitted_model,'automl_best_model.txt')

['automl_best_model.txt']

In [22]:
#Printing the best estimator for prediction from AutoML
print(fitted_model._final_estimator)

PreFittedSoftVotingRegressor(estimators=[('0',
                                          Pipeline(memory=None,
                                                   steps=[('maxabsscaler',
                                                           MaxAbsScaler(copy=True)),
                                                          ('lightgbmregressor',
                                                           LightGBMRegressor(boosting_type='gbdt',
                                                                             class_weight=None,
                                                                             colsample_bytree=1.0,
                                                                             importance_type='split',
                                                                             learning_rate=0.1,
                                                                             max_depth=-1,
                                                                  

In [23]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
incidents-exp,AutoML_5b4f94d5-e680-469e-890b-774eb1fb462b_22,,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [24]:

best_run.id

'AutoML_5b4f94d5-e680-469e-890b-774eb1fb462b_22'

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [26]:

run_metrics = best_run.get_metrics()
run_metrics

{'normalized_median_absolute_error': 0.0029812170226194193,
 'normalized_root_mean_squared_log_error': 0.04800541705810593,
 'r2_score': 0.5031106785977179,
 'mean_absolute_error': 126.72290595249338,
 'root_mean_squared_error': 377.85460168257146,
 'normalized_root_mean_squared_error': 0.04613261944248049,
 'root_mean_squared_log_error': 0.4325704069108028,
 'median_absolute_error': 24.41800583241699,
 'explained_variance': 0.5031535130951525,
 'normalized_mean_absolute_error': 0.015471717345559253,
 'spearman_correlation': 0.822950582687059,
 'mean_absolute_percentage_error': 89.88796114656861,
 'residuals': 'aml://artifactId/ExperimentRun/dcid.AutoML_5b4f94d5-e680-469e-890b-774eb1fb462b_22/residuals',
 'predicted_true': 'aml://artifactId/ExperimentRun/dcid.AutoML_5b4f94d5-e680-469e-890b-774eb1fb462b_22/predicted_true'}

In [ ]:
#TODO: Save the best model

## Model Deployment

## Model Deployment



## Register Model

In [29]:

#Model = best_run.register_model(model_name = "automl_best_model.pkl", model_path = './outputs/')


# az ml model register -n sklearn_mnist  --asset-path outputs/sklearn_mnist_model.pkl  --experiment-name myexperiment --run-id myrunid --tag area=mnist

from azureml.core import Model
from azureml.core.resource_configuration import ResourceConfiguration

model = Model.register(workspace=ws,
                       model_name='automl_best_model',                # Name of the registered model in your workspace.

                       model_path='./automl_best_model.txt',  # Local file to upload and register as a model.
                       model_framework=Model.Framework.SCIKITLEARN,
                       
                       description='linear regression model to predict time to close.')

print('Name:', model.name)
print('Version:', model.version)

Registering model automl_best_model
Name: automl_best_model
Version: 1


In [30]:
#Create inference config
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig

env = Environment.get(workspace=ws,name="AzureML-AutoML")
inferencefinal = InferenceConfig(entry_script='score.py',environment=env) 

## Deploy AutoML Model

### AutoML model resulted in better R2 score compared to HD model 
### So AUtoML is chosen for deployment

In [31]:

#Deploy model and check service state

from azureml.core import Model
from azureml.core.webservice import AciWebservice, Webservice, LocalWebservice

aci_config = AciWebservice.deploy_configuration()

# (cpu_cores=1, memory_gb=2,enable_app_insights=True, auth_enabled=True)  
service_name = 'deployautoml-6'

service = Model.deploy(ws, service_name, [model], deployment_config=aci_config, inference_config=inferencefinal, overwrite=True)
service.wait_for_deployment(show_output = True)
print(service.get_logs())


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running........................................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
2021-01-28T07:16:10,120564626+00:00 - gunicorn/run 
2021-01-28T07:16:10,220956800+00:00 - rsyslog/run 
2021-01-28T07:16:10,221031901+00:00 - iot-server/run 
2021-01-28T07:16:10,320856668+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_20a8278aa8b20dd48cc50f56a6d2586c/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_20a8278aa8b20dd48cc50f56a6d2586c/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_20a8278aa8b20dd48cc50f56a6d2586c/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/us

TODO: In the cell below, send a request to the web service you deployed to test it.

In [32]:
print(service)

print("********************************************")
print(service.get_logs())
#print(model)


AciWebservice(workspace=Workspace.create(name='quick-starts-ws-136346', subscription_id='a24a24d5-8d87-4c8a-99b6-91ed2d2df51f', resource_group='aml-quickstarts-136346'), name=deployautoml-6, image_id=None, compute_type=None, state=ACI, scoring_uri=Healthy, tags=http://33f7a1ec-34ba-45e8-80e5-61333e520460.southcentralus.azurecontainer.io/score, properties={}, created_by={'hasInferenceSchema': 'False', 'hasHttps': 'False'})
********************************************
2021-01-28T07:16:10,120564626+00:00 - gunicorn/run 
2021-01-28T07:16:10,220956800+00:00 - rsyslog/run 
2021-01-28T07:16:10,221031901+00:00 - iot-server/run 
2021-01-28T07:16:10,320856668+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_20a8278aa8b20dd48cc50f56a6d2586c/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_20a8278aa8b20dd48cc50f56a6d2586c/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/

## Create Webservice

In [33]:
services = Webservice.list(ws)
print(services[0].scoring_uri)
print(services[0].swagger_uri)
print(service.state)

http://33f7a1ec-34ba-45e8-80e5-61333e520460.southcentralus.azurecontainer.io/score
http://33f7a1ec-34ba-45e8-80e5-61333e520460.southcentralus.azurecontainer.io/swagger.json
Healthy


In [34]:
    scoring_uri = service.scoring_uri


In [35]:
print(service.scoring_uri)

http://33f7a1ec-34ba-45e8-80e5-61333e520460.southcentralus.azurecontainer.io/score


In [36]:
print(service.state)

Healthy


In [37]:
print("Swagger URI: " + service.swagger_uri)
#print("Authetication Key: " + service.get_keys()[0])

Swagger URI: http://33f7a1ec-34ba-45e8-80e5-61333e520460.southcentralus.azurecontainer.io/swagger.json


### Dataset totest

In [38]:
df1[1000:1002].T

,6738,6747
category,20.00,32.00
assignment_group,60.00,18.00
reopen_count,0.00,0.00
made_sla,0.00,0.00
time_to_close,124.80,515.45
priority_1 - Critical,0.00,0.00
priority_2 - High,1.00,0.00
priority_3 - Moderate,0.00,1.00
priority_4 - Low,0.00,0.00
impact_1 - High,1.00,0.00


## Webservice is tested for prediction

In [39]:
import requests
import json as jsn

# URL for the web service
# scoring_uri = 'http://29e036ea-6c5a-48e6-ad64-5464eb02ec6c.southcentralus.azurecontainer.io/score'


data = {'data':[
    {   
   'category':20.00,
    'assignment_group':60.00,
    'reopen_count':	0.00,
    'made_sla':	0.00,
    'category':20.00,
    'assignment_group':60.00,
    'reopen_count':0.00,
    'made_sla':0.00,
    'priority_1 - Critical':0.00,
    'priority_2 - High':1.00,
    'priority_3 - Moderate':0.00,
    'priority_4 - Low':0.00,
    'impact_1 - High':1.00,
    'impact_2 - Medium':0.00,
    'impact_3 - Low':0.00,
    'urgency_1 - High':0.00,
    'urgency_2 - Medium':1.00,
    'urgency_3 - Low':0.00,
    'closed_code_?':0.00,
    'closed_code_code 1':0.00,
    'closed_code_code 10':0.00,
    'closed_code_code 11':0.00,
    'closed_code_code 12':0.00,
    'closed_code_code 13':0.00,
    'closed_code_code 14':0.00,
    'closed_code_code 15':0.00,
    'closed_code_code 16':0.00,
    'closed_code_code 17':0.00,
    'closed_code_code 2':0.00,
    'closed_code_code 3':0.00,
    'closed_code_code 4':0.00,
    'closed_code_code 5':0.00,
    'closed_code_code 6':1.00,
    'closed_code_code 7':0.00,
    'closed_code_code 8':0.00,
    'closed_code_code 9':0.00
    }]
    }

input_data = jsn.dumps(data)
headers = {'Content-Type':'application/json'}

# # Make the request and display the response
result = requests.post(scoring_uri,input_data,headers=headers)
print("prediction:",result.text)

prediction: [243.99251671240256]


## Webservice Logs

In [40]:
    print(service.get_logs())

2021-01-28T07:16:10,120564626+00:00 - gunicorn/run 
2021-01-28T07:16:10,220956800+00:00 - rsyslog/run 
2021-01-28T07:16:10,221031901+00:00 - iot-server/run 
2021-01-28T07:16:10,320856668+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_20a8278aa8b20dd48cc50f56a6d2586c/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_20a8278aa8b20dd48cc50f56a6d2586c/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_20a8278aa8b20dd48cc50f56a6d2586c/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_20a8278aa8b20dd48cc50f56a6d2586c/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_20a8278aa8b20dd48cc50f56a6d2586c/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
rsyslogd

TODO: In the cell below, print the logs of the web service and delete the service

In [41]:

# service.delete()
# compute_target.delete()